<a id="introduction"></a>
# The Event-Related Potential *for the practicing neuroscientist*

<div class="question">
    
_**Synopsis**_ 

**Data:** 1 s of scalp EEG data sampled at 500 Hz during 1,000 trials in two conditions.

**Goal:** Characterize the response of the EEG in the two conditions.

**Tools:** Visualization, event-related potential, confidence intervals, bootstrapping.
</div>

* [Background](#background)
* [Case Study Data](#case-study-data)
* [Data Analysis](#data-analysis)
    * [Visual Inspection](#visual-inspection)
    * [Plotting the ERP](#plotting-the-erp)
    * [Confidence Intervals for the ERP (Method 1)](#cis-m1)
    * [Comparing ERPs](#comparing-erps)
    * [Confidence Intervals for the ERP (Method 2)](#cis-m2)
    * [A Bootstrap Test to Compare ERPs](#bootstrap)
* [Summary](#summary)
    
    

Before we start any computations, let's import some modules and functions that we will use throughout the chapter. A module can be imported any time, but there are a few things that we know we will need straight off the bat. For clarity, it is best practice to import packages at the beginning. 

You will see that we have imported `matlab.pyplot` and can call any functions in the module with `plt.f()`, where `f` should be replaced with the name of the desired function. Hence, if we want to plot something, we could call `plt.plot()`. However, we will use that function so often that it will be convenient to import `plot()` directly without typing `plt.` first. 

In [ ]:
from scipy.io import loadmat       # Import function to read data.
from IPython.lib.display import YouTubeVideo  # Enable YouTube videos
import numpy as np                 # Import numpy for computations
import matplotlib.pyplot as plt    # Import a useful plotting package, 
from matplotlib.pyplot import plot, xlabel, ylabel, title, show, subplots, savefig
                                   # ... and a few specific functions that are used often
# Tools for this chapter
from matplotlib.pyplot import imshow, colorbar, hlines, vlines
from numpy import sqrt
from numpy.random import randint


## On-ramp: computing the event-related potential in Python

We begin this module with an "*on-ramp*" to analysis. The purpose of this on-ramp is to introduce you immediately to a core concept in this module: how to compute an event-related potential with error bars in Python. You may not understand all aspects of the program here, but that's not the point. Instead, the purpose of this on-ramp is to  illustrate what *can* be done. Our advice is to simply run the code below and see what happens ...

In [ ]:
# Load and display info on the data: 
data = loadmat('EEG-1.mat') # `loadmat` returns a dict
# view the keys
print('The keys in `data` are: \n  ' + str(data.keys()))
# There are two sets of EEGs for two conditions: EEGa, and EEGb.
# Pull out `EEGa` and `t` from `data`, and view the shape of these variables.
eeg_a = data['EEGa']
t = data['t'][0]
print('The shape of `t` is: ' + str(t.shape))
print('The shape of `eeg_a` is: ' + str(eeg_a.shape))
# get the number of trials by looking at the number of rows in `eegA`
ntrials = len(eeg_a) # this is equivalent to `eegA.shape[0]`
print('The number of trials in `eeg_a` is: ' + str(ntrials))

In [ ]:
# Compute the ERP:
mn_a = eeg_a.mean(0)  # Compute the mean signal across trials for `eegA` (this is the ERP)
sd_a = eeg_a.std(0)  # Compute the std of the signal across trials
sd_mn_a = sd_a / sqrt(ntrials)  # Compute the std of the mean

# Ensure the mean was computed across trials for each time point (not across time for each trial):
if (mn_a.size == t.size):
    print('The length of the mean of `eeg_a` is ' + str(t.size) + ' timepoints')
else:
    print('The mean of `eeg_a` was not calculated correctly')
        

In [ ]:
# Visualize the ERP with 95% confidence intervals:
fig1, ax1 = plt.subplots()
fig1.set_size_inches(12,4)
# Plot ERP in default color, and plot upper and lower bounds
# for 95% confidence interval in black
ax1.plot(t, mn_a, lw=3) # ERP
ax1.plot(t, mn_a + 2 * sd_mn_a, 'k:', lw=1) # Upper confidence interval bound
ax1.plot(t, mn_a - 2 * sd_mn_a, 'k:', lw=1) # Lower confidence interval bound
# Add labels:
ax1.set_xlabel('Time [s]')
ax1.set_ylabel('Voltage [$\mu$ V]')
ax1.set_title('ERP of condition A')

<div class="question">
    
**Q:** Try to read the code above. Can you see how it loads data, computes the event-related potential and error, and then plots the results?

**A:** If you've never computed an event-related potential before, that's an especially difficult question. Please continue on to learn this **and more**!
</div>

## Background <a class="anchor" id="background"></a>

In [ ]:
YouTubeVideo('Cy_BF7smAkk')
# NO CODE

Voltage recordings from the scalp surface - the electroencephalogram or EEG - provide a powerful window into brain voltage activity.  Some of the earliest human EEG recording occurred in 1924, when [Dr. Hans Berger](https://en.wikipedia.org/wiki/Hans_Berger) made a remarkable discovery:  the EEG of a human subject at rest with eyes closed exhibits rhythmic activity, an approximately 10 Hz oscillation he labeled the alpha rhythm.  Although now studied for nearly 100 years, the definitive functional role (if any) of the alpha rhythm remains unknown.  Since then, many other EEG rhythms have been detected and labelled (typically with Greek letters) and the analysis of EEG rhythms remains [an active area of research](https://global.oup.com/academic/product/rhythms-of-the-brain-9780199828234). 

Compared to other modalities for measuring brain activity, the EEG possesses both advantages and disadvantages.  Perhaps the most important advantages are:

1. The EEG is non-invasive, and
2. The EEG permits a high temporal resolution (on the order of milliseconds).

However, the EEG measure also suffers from significant disadvantages, the most devastating being the poor spatial resolution;  a single scalp electrode detects the summed activity from approximately 10 cm<sup>2</sup> of cortex.

In this chapter, we consider EEG data recorded from a single scalp electrode.  We will analyze these data to determine what (if any) activity is evoked following two different types of stimuli presented to a human subject.  In doing so, we will use Python, and see how this powerful tool can help us understand these time series data.  We begin with a brief description of the EEG data.

[Return to top](#introduction)

## Case Study: an EEG ERP task <a class="anchor" id="case-study-data"><a/>

In [ ]:
YouTubeVideo('q2-DjvPRaNA')
# NO CODE

An undergraduate student volunteers to participate in a psychology study at his university. In this study, EEG electrodes (sampling rate 500 Hz, i.e., 500 samples per second) are placed on the student's scalp, and he is seated in a comfortable chair in a dark, electrically isolated room.  The student is instructed to place headphones over his ears and listen to a series of repeated sounds.  The sounds consist of two tones - either a high pitch tone or a low pitch tone.  A single tone is presented once every few seconds, and the student responds with a button press to the low pitch tone.  The tone presentation is repeated to collect the EEG response to numerous presentations of the two tones, as illustrated here:

<img src="imgs/2-1.png">

In this cartoon illustration of the EEG experiment, the EEG electrodes are placed on the scalp surface of a human subject (left).  The EEG activity (blue) is recorded as a function of time during presentation of high pitch tones (black) and low pitch tones (orange).

Our collaborator leading this research study has agreed to provide us with EEG data recorded at a single electrode for 1000 presentations of the high pitch tone, and 1000 presentations of the low pitch tone.  In each presentation - or "trial" - she provides us with 1 s of EEG data, such that the tone occurs at 0.25 s into the trial.  She asks us to analyze these data to determine whether the EEG signal differs following the two tone presentations.

[Return to top](#introduction)

### Visual Inspection <a id="visual-inspection"></a>

In [ ]:
YouTubeVideo('uSjd41G-yNY')
# NO CODE

`eeg_a` is a complicated variable that contains many elements. To understand this data, we might attempt to read the values contained in each element. For example, we can print out the EEG data for the first trial of condition A

In [ ]:
print(eeg_a[0])  # eeg_a[0, :] is equivalent to eeg_a[0]

In this command, we index the first row of the matrix `EEGa` and print out all columns (corresponding to all moments of time).

<div class="question">

**Q.** Upon issuing this command what do you find? Does the printout help you understand these data?

**A.** You should observe a list of 500 numbers that begins 

    `-1.85909632e-01   4.49876010e-01   1.06070801e+00  -4.71265246e-01   1.68669327e+00   9.38221338e-01 ...`
    
We might conclude that these numbers exhibit variability (i.e., the values are both positive and negative), but examining the data in this way is not particularly useful. For example, determining trends in the behavior (such as intervals of repeated activity) through inspection of these printed numbers alone is extremely difficult.
</div>

In [ ]:
YouTubeVideo('9qx29zDxcAc')
# NO CODE

Printing out the data to the screen is **not useful** in this case. How else can we deepen our understanding of these data? Let’s make a plot:
<a id="fig:2"></a>

In [ ]:
fig2, ax2 = plt.subplots()
ax2.plot(eeg_a[0])                   
# we can save the figure in the 'imgs' folder with a number that matches the textbook, if we want
# fig2.savefig('imgs/2-2-a.png')

Visualizing the data in this way, we immediately notice many features. First, let’s consider the axes. The horizontal axis extends from 0 to (nearly) 500. This corresponds to the 500 columns in the variable `eeg_a`. While this visualization is useful, it would be more informative to plot the EEG data as a function of time rather than indices. Fortunately, we possess a variable `t` in the workspace that corresponds to the time axis. Determining the size of the variable `t`, we find it is a vector with 1 row and 500 columns. Each column corresponds to a point in time. 

<div class="question">
    
**Q.** Plot the variable `t`. What is its range? 
</div>

__A__:

In [ ]:
fig3, ax3 = plt.subplots()
ax3.plot(t, eeg_a[0]) 

The variable `t` corresponds to the 1 s of EEG data recorded in each trial. We can also use the variable `t` to determine the sampling interval,

In [ ]:
# quick and dirty approach, if we assume `t` is sampled at regular intervals
dt = t[1] - t[0]
# can also do:
dt = np.mean(np.diff(t))
print('The sampling interval is ' + str(dt) + 's')

The new variable `dt` corresponds to the time between samples.

<div class="question">

**Q.** What is the value of `dt`? We were told by our collaborator that the sampling frequency is 500 Hz. Is the value of `dt` consistent with this sampling frequency?

**A.** Yes, it is consistent. Using the command `print(dt)`, we find that `dt` is 0.002 s, or 2 ms. The sampling frequency of 500 Hz corresponds to one sample of the EEG data every 1/(500 Hz) = 2 ms. If the two were not consistent, we would return to our collaborator and figure out what has gone wrong. In general, it’s useful to ask such questions along the way to make sure we understand the formatting of the data and catch any potentially serious misunderstandings early in the analysis.
</div>

<div id="singleTrial">
    
We can now combine the time axis with the EEG data to make a more complete plot. Let’s also label the axes and give the plot a title.

</div>

In [ ]:
fig4, ax4 = plt.subplots()
ax4.plot(t, eeg_a[0]) 
ax4.set_xlabel('Time [s]')                   
ax4.set_ylabel('Voltage [$\mu$ V]')         
ax4.set_title('EEG data from condition A, Trial 1')

# Add a vertical line to indicate the stimulus onset time
stim_onset_time = 0.25
max_abs_V = np.max(np.abs(eeg_a))
ax4.vlines(stim_onset_time, -max_abs_V, max_abs_V, 'r', lw=2)
# we can save the figure in the 'imgs' folder with a number that matches the textbook, if we want
# fig4.savefig('imgs/2-2-c.png')

This plot provides a nice summary of the data in the first trial of condition A. Visual inspection of the plot suggests that these data exhibit complicated activity. We know from our collaborator that the stimulus occurs at time 0.25 s in each trial. Note how we indicated this time as a vertical line in the plot above. This command includes additional options that make the line red (`'red'`) and a bit wider (`lw=2`). 

<div class="question">
    
**Q.** What else, if anything, can you say about the single trial of EEG data plotted above? Does the visual inspection reveal any particular change in the EEG activity following the stimulus presentation?
</div>

So far we have visualized only the data from condition A. Because we are interested in whether the EEG behaves differently in the two conditions, visualizing both conditions simultaneously would be of use. We can do this as follows:
<a id="fig:3"></a>

In [ ]:
# Create a wider figure, and plot the first trial from both eeg_a and eeg_b:
eeg_b = eeg_b = data['EEGb']
fig5, ax5 = plt.subplots(figsize=(12, 3))
ax5.plot(t, eeg_a[0])           
ax5.plot(t, eeg_b[0])

# Add labels:
ax5.set_xlabel('Time [s]')             
ax5.set_ylabel('Voltage [\mu V]')       
ax5.set_title('EEG data across conditions, Trial 1')
ax5.legend(['eeg_a', 'eeg_b'])

# we can save the figure in the 'imgs' folder with a number that matches the textbook, if we want
# fig5.savefig('imgs/2-3.png')

<div class="question">
    
**Q.** Compare the voltage traces from the first trial of conditions A and B as plotted above. What similarities and differences do you observe?
</div>

__A__: Hard to see any particular differences here besides the fact that `eeg_b` has greater absolute amplitudes.

<div class="question">
    
**Q.** The analysis has so far focused only on the first trial. Repeat this visual inspection of the data for different trials. What do you find? What similarities and differences exist between the two conditions across trials?
</div>

__A__: A figure containing 10 subplots of trials 2-11 is generated by the code below. Again, hard to see any particular differences here besides the fact that `eeg_b` has greater absolute amplitudes in all of these trials.

In [ ]:
# Create a subplot of the next 10 trials
fig6, ax6 = plt.subplots(5,2, figsize=(12,15))
fig6.set_tight_layout(True) # make sure there's no unnecessary overlap or whitespace between axes 

for trial in range(1,11):
    # Get the appropriate row, `r`, and column, `c` of the
    # subplot to plot to, depending on the current trial number.
    if np.mod(trial,2):
        r = int((trial-1)/2)
        c = 0
    else:
        r = int((trial/2)-1)
        c = 1
    
    # Plot the data and set labels.
    ax6[r][c].plot(t, eeg_a[trial])
    ax6[r][c].plot(t, eeg_b[trial])
    ax6[r][c].set_title('EEG data, trial ' + str(trial))
    ax6[r][c].legend(['eeg_a', 'eeg_b'])

In [ ]:
YouTubeVideo('nandZ5aaRaQ')
# NO CODE

These techniques allow us to visualize the data one trial at a time. That is useful but can be time consuming, especially for a large number of trials. For the EEG data of interest here, each condition contains 1,000 trials, and to visualize each trial separately could require 2,000 plots. We can certainly create 2,000 plots, but the subsequent visual inspection would be time consuming and difficult. Fortunately, a more efficient visualization approach exists: we can display the entire structure of the data across both time and trials as an image:
<a id="fig:4"></a>

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

# Plot all trials of `eeg_a` and `eeg_b` in two colormaps:
fig7, ax7 = plt.subplots(2,1, figsize=(12,6))
fig7.set_tight_layout(True)
ax7[0].imshow(eeg_a,
           extent=[t[0], t[-1], 0, ntrials], # specify axes lims      
           aspect='auto') # specify aspect ratio
c_bar_map = ax7[1].imshow(eeg_b,
           extent=[t[0], t[-1], 0, ntrials],       
           aspect='auto')
# We want to use one set of common labels for both axes.
# To do this, create one big, hidden subplot, and give
# this subplot labels. 
hid_ax = fig7.add_subplot(1,1,1, frameon=False)
hid_ax.tick_params(labelcolor='none', length = 0)
hid_ax.set_ylabel('Trial #')
hid_ax.set_xlabel('Time (s)')

# Decrease tick labels of `ax` so they don't interfere with
# labels of `hid_ax`.
ax7[0].tick_params(labelsize='x-small') # this can be string or numeric
ax7[1].tick_params(labelsize='x-small')

# Add a colorbar to a separate axes on the figure, and add hidden axes
# next to `ax[0]` and `ax[1]` to ensure colorbar doesn't overlap with them.
divider_hid_ax = make_axes_locatable(hid_ax) # divider to create the colorbar axes
divider_ax0 = make_axes_locatable(ax7[0]) # divider to create hidden axes next to `ax[0]`
divider_ax1 = make_axes_locatable(ax7[1]) # divider to create hidden axes next to `ax[1]`
hid_ax0 = divider_ax0.append_axes('right', size='5%', pad='5%')
hid_ax1 = divider_ax1.append_axes('right', size='5%', pad='5%')
hid_ax0.set_visible(False)
hid_ax1.set_visible(False)
cax = divider_hid_ax.append_axes('right', size='5%', pad='5%')
# Add colorbar, with values mapped to the image on `ax[1]` (instead of `[ax0]`, because
# `eeg_b` contains greater abs values), and an appropriate label
cbar = fig7.colorbar(c_bar_map, cax=cax)
cbar.set_label('Voltage ($\mu$ V)', rotation=-90)

# Add lines for stimulus onset times
ax7[0].vlines(stim_onset_time, 0, ntrials, 'r', lw=2, label='stim onset time')
ax7[1].vlines(stim_onset_time, 0, ntrials, 'r', lw=2)
ax7[0].legend()

# we can save the figure in the 'imgs' folder with a number that matches the textbook, if we want
# fig7.savefig('imgs/2-4.png')

The `imshow` command allows us to visualize the entire matrix `eeg_a` as a function of trial number and time. Each row corresponds to a single trial of duration 1 s, and the color indicates the voltage, with darker (lighter) colors indicating lower (higher) voltages. This plot also indicates the time of stimulus presentation with a vertical red line as a cue to assist visual inspection.

<div class="python-note">
    
We have used the default color map for the plot above. There are many other options; use `plt.colormaps?` for details.
</div>

<div class="question">
    
**Q.**
Upon close inspection of the figure above, what response, if any, do you observe following the stimulus presentation? (Look *really* carefully.) Repeat this visualization and analysis for `EEGb`. How do the two conditions compare?
</div>

__A__: For `eeg_a`, a thin band of lower voltage following stimulus presentation, centered around 0.35s from t=0.

[Return to top](#introduction)

### Plotting the ERP <a id="plotting-the-erp"></a>

In [ ]:
YouTubeVideo('kPr2GLSKLJg')
# NO CODE

Visual inspection of the EEG data has so far come up empty. The EEG traces appear noisy or perhaps rhythmic, but from visual inspection of the individual trials it’s difficult to make a decisive conclusion of underlying structure (<a href="#singleTrial">see above</a>). To further investigate the activity in these data, we compute the **event-related potential** (ERP).

To compute the ERP, we first assume that each trial evokes an instantiation of the same underlying brain process. So, in this case, we assume that the same brain response is evoked 1,000 times (once for each trial) for each condition. However, the evoked response due to the stimulus is small and hidden in the EEG signal by other ongoing activity unrelated to the stimulus (e.g., daydreaming, thoughts of dinner, thoughts of homework). Therefore, to tease out the weak evoked effect, **we average the EEG responses across trials**. Ideally, EEG activity unrelated to the stimulus will cancel out in the average, while EEG activity evoked by the stimulus will sum constructively. The procedure to perform and display this averaging can be done in Python as follows:
<a id="fig:5"></a>

In [ ]:
fig8, ax8 = plt.subplots(figsize=(12,3))
erp_line = ax8.plot(t, eeg_a.mean(0))
ax8.set_xlabel('Time (s)')           
ax8.set_ylabel('Voltage ($\mu$ V)')
ax8.set_title('ERP of condition A')  
# we can save the figure in the 'imgs' folder with a number that matches the textbook, if we want
# fig8.savefig('imgs/2-5.png')

In the first line, we compute the mean of `eeg_a`. The default behavior is to compute the mean of all elements of the array. By calling `eeg_a.mean(0)`, we compute the mean along the zeroth dimension (in this case, across all trials). The result is the ERP for condition A.

<div class="question">
    
**Q.** Consider the ERP for condition A plotted above. Update this figure to include a vertical line at the location of the stimulus, and the ERP for condition B. How, if at all, do the ERPs for Conditions A and B differ?

</div>

__A__:

In [ ]:
# Plot the ERPs of `eeg_a` and `eeg_b`:
erp_a = eeg_a.mean(0)
erp_b = eeg_b.mean(0)
max_erp_V = np.max([erp_a, erp_b])
fig9, ax9 = plt.subplots(2,1, figsize=(12,6))
fig9.set_tight_layout(True)
ax9[0].plot(t, erp_a)
ax9[1].plot(t, erp_b)

# We want to use one set of common labels for both axes.
# To do this, create one big, hidden subplot, and give
# this subplot labels. 
hid_ax = fig9.add_subplot(1,1,1, frameon=False)
hid_ax.tick_params(labelcolor='none', length = 0)
hid_ax.set_ylabel('Voltage ($\mu$ V)')
hid_ax.set_xlabel('Time (s)')
hid_ax.set_title('ERPs across conditions: A (top plot), B (bottom plot)')

# Add lines for stimulus onset times
ax9[0].vlines(stim_onset_time, -max_erp_V, max_erp_V, 'r', lw=2, label='stim onset time')
ax9[1].vlines(stim_onset_time, -max_erp_V, max_erp_V, 'r', lw=2)
ax9[0].legend()

The ERP of condition A shows the mean voltage across trials at each moment in time. Visual inspection suggests that before stimulus presentation (i.e., times 0 s to 0.25 s) the EEG fluctuates around zero. Then, after stimulus presentation, the ERP increases and decreases substantially above and below zero. Which, if any, of these deviations following stimulation are significant? To address this, we make use of the trial structure of the EEG data to compute confidence bounds for the ERP. We do so in two ways.

[Return to top](#introduction)

### Confidence Intervals for the ERP (Method 1) <a id="cis-m1"></a>

In [ ]:
YouTubeVideo('pXCJbyrw8Ug')
# NO CODE

To compute the ERP we average the EEG data across many trials. Because of this, we may make use of a powerful theorem in statistics—the [*central limit theorem*](https://en.wikipedia.org/wiki/Central_limit_theorem) (CLT)—to include approximate confidence bounds in the ERP figure. Briefly, this theorem states that the mean of a sufficiently large number of independent random variables, each with finite mean and variance, will be approximately [normally distributed](https://en.wikipedia.org/wiki/Normal_distribution). Remember that the ERP at each moment in time is the sum of EEG activity across trials (then scaled by a constant, the number of trials). Let’s assume that the trials are independent (i.e., one trial does not depend on any other trial). Let’s also assume that the EEG data at each moment in time have finite mean and variance. With those assumptions, we have satisfied the CLT and may therefore conclude that the ERP at each moment in time is approximately normally distributed.

<div class="question">
    
**Q.** To use the CLT, we make two assumptions about the EEG data. Are these assumptions reasonable?

**A.** We assume that the EEG data are independent across trials. This assumption may fail if, for example, the activity in one trial influences the activity in the next trial. We also assume that the EEG data are “well-behaved” (i.e., have finite mean and variance). That is a reasonable assumption for physical data we observe from the brain; we expect the EEG data to always remain finite and not diverge to plus or minus infinity.

</div>

This conclusion—that the ERP at each moment in time is approximately normally distributed—is useful because the normal distribution (also known as the Gaussian distribution or bell curve) possesses many convenient properties. First, a normal distribution is relatively simple; it can be completely specified with two parameters: the mean value and the standard deviation. Second, 95% of the values drawn from a normal distribution lie within approximately two standard deviations of the mean.

<img src="imgs/gaussian.png" alt="Example Gaussian" style="width:40%; max-width:300px;"/>

Here's a plot of the canonical normal distribution showing the mean (dotted vertical line) and standard deviation (blue). Ninety-five percent of values lie within the interval indicated by the red bar. 

Therefore, to construct a 95% confidence interval for the ERP, we need to determine the mean and standard deviation of the mean across trials at each point in time. To compute the mean in Python is easy:

In [ ]:
mn_a = eeg_a.mean(0)

Note that when we refer to the *mean* here we could instead write *sample mean* because we use the observed data to estimate the theoretical mean that we would see if we kept repeating this experiment. This distinction is not essential to our goals here, but it is important when talking to your statistics-minded colleagues. Throughout the book, we omit the term sample when referring to sample means, variances, covariances, and so forth, unless this distinction is essential to the discussion.

To compute the standard deviation of the mean, we start by computing the standard deviation of the data:

In [ ]:
sd_a = eeg_a.std(0)

But we’re not interested in the standard deviation of the EEG data across trials; instead, we’re interested in the standard deviation *of the estimate of the mean*. To calculate the standard deviation of the mean, we divide the standard deviation of the data by the square root of the number of trials (i.e., the number of terms used to compute the mean). In Python,

In [ ]:
sd_mn_a = sd_a / sqrt(ntrials)  # Compute the std of the mean

Now, having found the mean (`mn_a`) and the standard deviation of the mean (`sd_mn_a`), we can compute a 95% confidence interval for the ERP. We again exploit the observation, based on the central limit theorem, that the ERP is normally distributed at each instant of time. With these calculations, we can plot the ERP and the 95% confidence interval:

In [ ]:
fig10, ax10 = subplots(figsize=(12, 3)) 
ax10.plot(t, mn_a, lw=2)              # Plot the ERP of condition A
ax10.plot(t, mn_a + 2 * sd_mn_a, 'k:', lw=1)  # plot the upper CI bound
ax10.plot(t, mn_a - 2 * sd_mn_a, 'k:', lw=1)  # plot the lower CI bound
ax10.set_xlabel('Time [s]')                     # Label the axes
ax10.set_ylabel('Voltage [$\mu$ V]')
ax10.set_title('ERP of condition A with a 95% confidence interval')

The ERP computed with confidence intervals allows us to ask specific questions about the data. For example, does the ERP ever differ significantly from zero? To answer this, we look for intervals of the ERP for which the confidence intervals do not include zero. To aid visual inspection, we add to the ERP plot a horizontal line at 0: <a id="plt:erpA-m1"></a>

In [ ]:
# add a horizontal line, additional xticks, and a grid
ax10.hlines(0, t[0], t[-1], 'r')
ax10.set_xticks(np.arange(0,1.05,0.05))
ax10.grid(True)
fig10
# we can save the figure in the 'imgs' folder with a number that matches the textbook, if we want
# fig10.savefig('imgs/2-7.png')

<div class="python-note">
    
A good rule of thumb when you are programming is that you should not be rewriting (or copy-pasting) code over and over again. Instead, you should write a function that you can call whenever you need the action that you keep repeating. At this point, we have resized the plots and labeled the axes the same way several times so we should fix the default plot size and write a function that automates the labeling so that next time we make a plot, we don't need to rewrite the same code again.
</div>

In [ ]:
# Change the default figure size
from matplotlib import rcParams
rcParams['figure.figsize'] = (12, 3)

# Create a function to label plots
def labelPlot(ax, title='Title'):
    '''
    Labels the x-axis as 'Time [s]' and the y-axis as 'Voltage [$\mu V$]'.
    
    Parameters:
    -----------
        ax: the axes handle to label
        title: string variable to be used as
                      the plot title (default: 'Title')
                       
    '''
    ax.set_xlabel('Time (s)')           # x-axis is time
    ax.set_ylabel('Voltage ($\mu$ V)')  # y-axis is voltage
    ax.set_title(title)          # use the input here

<div class="question">
    
**Q.** How would you write a function to compute and return the ERP and confidence bounds of a dataset, and plot the result?

</div>

__A__:

In [ ]:
import math
import numpy as np

def plot_erp(data):
    ''' 
    Plots an ERP with 95% confidence intervals, and returns a dict of the ERP,
    confidence intervals, and generated figure for a dataset, `data`.
    
    Parameters:
    -----------
        data: a ndarray, 'm' (# of trials) by 'n' 
              (# of time points), of voltage data.
    
    Returns:
    --------
        data_out: a dict containing the computed ERP, confidence intervals, and
                  generated figure based on `data`
    '''
    # Compute ERP and CIs:
    m = data.shape[0]
    n = data.shape[1]
    erp = data.mean(0)
    sd_erp = data.std(0) / sqrt(m)
    upper_ci = erp + (2 * sd_erp)
    lower_ci = erp - (2 * sd_erp)
    
    # Plot ERP and CIs:
    fig, ax = subplots()
    erp_plot = ax.plot(np.arange(0,n), erp, lw=2)
    ax.plot(np.arange(0,n), upper_ci, 'k:', lw=1)
    ax.plot(np.arange(0,n), lower_ci, 'k:', lw=1)
    data_out = {'erp': erp, 'upper_ci': upper_ci, 'lower_ci': lower_ci, 'figure': fig, \
            'erp_plot':erp_plot}
    return data_out

In [ ]:
# Example using the function we define above:
erp_data_a = plot_erp(eeg_a)
ax = plt.gca()
labelPlot(ax, 'ERP for Condition A with 95% Confidence Intervals')
ax.set_xlabel('Time Samples') # rename xlabel since it's not in seconds

In the figure above, the thick line indicates the ERP for Condition A (i.e., the mean of the EEG across trials) while the thin dotted lines indicate the 95% confidence intervals.

We find three time intervals at which the confidence intervals of the ERP do not include zero: near 0.27 s, near 0.37 s, and near 0.47 s. These results suggest that for an interval of time following the stimulus presentation in condition A, the observed ERP is not a random fluctuation about zero but instead contains consistent structure across trials.

<div class="question">
    
**Q.** Construct the ERP with confidence intervals for condition B. As for condition A, you should find that before stimulus presentation the ERP fluctuates around zero. What intervals of time, if any, differ significantly from zero?
</div>

__A__:

In [ ]:
erp_data_b = plot_erp(eeg_b)
# change ERP line plot to red
l = erp_data_b['erp_plot']
l[0].set_color('r')
ax = plt.gca()
labelPlot(ax, 'ERP for Condition B with 95% Confidence Intervals')
ax.set_xlabel('Time Samples') # rename xlabel since it's not in seconds

[Return to top](#introduction)

### Comparing ERPs <a class="anchor" id="comparing-erps"></a>

To facilitate further inspection of the data, we compute the difference between the ERPs in the two conditions. In the differenced signal, large deviations between the two conditions will appear as large differences from zero. To determine whether a deviation is significantly different from zero, we need to determine the confidence interval for the differenced ERP. This requires we propagate the standard deviation of the mean for both ERPs to the new differenced ERP. The propagated standard deviation of the mean at a fixed moment in time is computed as,

<a id="eq:1"></a>
$$ \sigma = \sqrt{\frac{\sigma_A^2}{K} + \frac{\sigma_B^2}{K}}, \tag{1}$$

where $\sigma_A$ is the standard deviation of the data from condition A, $\sigma_B$ is the standard deviation of the data from condition B, and $K$ is the number of trials. In Python we compute the differenced ERP and standard deviation of the mean of the difference as follows:

<a id="plt:differencedERP"></a>

In [ ]:
# Calculate differenced ERP and its standard deviation
erp_diff = erp_data_a['erp'] - erp_data_b['erp']
sd_mn_diff = sqrt( ((erp_data_a['erp'] / sqrt(ntrials)) ** 2) + \
                   ((erp_data_b['erp'] / sqrt(ntrials)) ** 2) )

# Plot the differenced ERP and its CIs, and a horizontal line
# at 0
fig11, ax11 = plt.subplots()
ax11.plot(t, erp_diff, lw=2)
ax11.plot(t, erp_diff + 2 * sd_mn_diff, 'k:')  
ax11.plot(t, erp_diff - 2 * sd_mn_diff, 'k:')  
ax11.hlines(0, t[0], t[-1], 'r')
ax11.set_xticks(np.arange(0,1.05,0.05)) # set xticks so we can see time intervals more clearly
ax11.grid(True)
labelPlot(ax11, 'Differenced ERP')

<div class="question">
    
**Q:** Examine the plot of the differenced ERP. In what intervals of time do the EEG responses in the two conditions significantly differ?
</div>

__A__: Roughly, between 0.27-0.31s, 0.32-0.36s, 0.37-0.40s, 0.43-0.46s, 0.47-0.49s 

[Return to top](#introduction)

### Confidence Intervals for the ERP (Method 2) <a id="cis-m2"></a>

In [ ]:
YouTubeVideo('vVXH4XsPFEs')
# NO CODE

So far we have computed confidence intervals for the ERPs by relying on the central limit theorem and approximating the average voltage values at each point in time as normally distributed. That’s a completely reasonable approach. And because the normal distribution is so well-behaved, it’s easy to compute the 95% confidence intervals. An alternative approach to generate confidence intervals is through a **bootstrap** procedure. Bootstrapping is a resampling method that allows us to estimate the sampling distribution of many different statistics. In this chapter, we implement a *nonparametric bootstrap* (see note). To do so, we generate new *pseudodata* from the observed EEG data. We begin by using a bootstrapping procedure to create confidence intervals for the ERPs observed in each condition.

<div class="math-note">
    
**A note on the nonparametric bootstrap.** Briefly, there is strong theoretical justification for the nonparametric bootstrap. The fundamental idea is that resampling the data with replacement is equivalent to sampling new pseudodata from the empirical cumulative distribution function (eCDF) of the observed data. For a large sample of independent, identically distributed random variables, the distribution of the pseudodata generated from the eCDF will be close to the true distribution of the data. Note the important caveat that the variables are independent, identically distributed; this assumption fails in many cases, such as for time series. Here, we assume that each trial is drawn independently from the same distribution (i.e., the trials are independent, identically distributed variables).
</div>

We implement the bootstrapping procedure to compute pointwise confidence intervals. By pointwise we mean that the confidence intervals are computed separately for each point in time, and interactions across time are not considered. The prescription for the bootstrapping procedure follows four steps:

1. Sample with replacement 1,000 trials of the EEG data from condition A.
1. Average these 1,000 trials to create a resampled ERP.
1. Repeat these two steps 3,000 times to create a distribution of ERPs.
1. For each time point, identify the values greater than 2.5% and less than 97.5% of all 3,000 values. This range determines the 95% confidence interval for the ERP for that time point.

Let’s now implement each step in Python.

In [ ]:
YouTubeVideo('mqDEJyW_z4c')
# NO CODE

**Step 1.** In step 1 we must sample with replacement from the EEG data. To visualize this procedure, imagine placing 1,000 marbles in an opaque bag. Each marble is assigned a unique integer value from 1 to 1,000. Now, reach your hand into the bag, grab a marble, record its number, and replace the marble in the bag. We assume that each marble is equally likely to be selected at each draw (i.e., there are no special features that allow some marbles to be drawn more often). Repeat this procedure 1,000 times to create a list of 1,000 integers. Notice that after recording the drawn marble’s number, we replace it in the bag. So, we could potentially draw the same marble 1,000 times, although that’s extremely unlikely. Performing this sampling with replacement procedure by hand would, of course, be extremely time consuming (e.g., who will paint integers on each marble?). Fortunately, Python provides a function to perform sampling with replacement:

In [ ]:
# Draw 1000 integers with replacement from [0, 1000)
i = np.random.randint(0, ntrials, size=ntrials)

The first and second inputs to `randint()` specify the minimum and maximum integers to draw, respectively. Note that the low number is included in the set, but the high number is not. If only the upper bound is given, the lower bound is assumed to be zero (i.e., we can rewrite the above line as `np.random.randint(ntrials, size=ntrials)`). The last input indicats the number of samples to draw (as always, use `np.random.randint?` to find out more).

<div class="question">
    
**Q.** Examine the values of `i`. What values do you find?
</div>

__A__: The result `i` provides a list of integers between 0 and 999. These values specify the trials to use in creating the resampled EEG. This resampled EEG will contain the same number of trials as the original EEG (i.e., 1,000 trials) but in a different order and with possibly repeated trials. For example, if the sampling with replacement procedure returns

    i = [10, 941, 3, 400, 10, ...

then the first and fifth trials of the resampled EEG will equal the tenth trial of the original EEG. We create the resampled EEG in Python as follows:

In [ ]:
eeg_0 = eeg_a[i]  # Create the resampled EEG.

In this code we use the variable `i` as the index to the rows of `eeg_a`.

<div class="question">
    
**Q.** What is the shape of the new variable `eeg_0`? Is this shape consistent with the original EEG datasets?
</div>

__A__: The shape is 1000,500. Yes, same as `eeg_a`

In [ ]:
print(eeg_0.shape)

That completes step 1 of the resampling procedure.

In [ ]:
YouTubeVideo('bUzuNojLUik')
# NO CODE

**Step 2.** This step is easy: we create a resampled ERP from the resampled EEG data. Computing the resampled ERP requires only one line of code in Python:

In [ ]:
erp_0 = eeg_0.mean(0)  # Create the resampled ERP

<div class="question">
    
**Q.** What is the difference between the resampled EEG and resampled ERP? Explain your answer in words.
</div>

__A__: The resampled ERP is the mean of the resampled ERP across the trials

<div class="question">
    
**Q.** Plot the resampled ERP that we created. What does it look like?
</div>

__A__: Below is code used to generate a plot of the resampled ERP. It looks very similar to our real ERP (plotted directly below, in red), but a bit noisier.

In [ ]:
# Plot original ERP after changing its color to red
# erp_line[0].set_color('r')
# fig8

# Plot resampled ERP
fig12, ax12 = plt.subplots(2,1, figsize=(12,6))
fig12.set_tight_layout(True)
ax12[0].plot(t, erp_0)
ax12[1].plot(t, erp_a, 'r')
ax12[0].set_xticks(np.arange(0,1.05,0.05))
ax12[0].grid(True)
ax12[1].set_xticks(np.arange(0,1.05,0.05))
ax12[1].grid(True)
labelPlot(ax12[0], 'Resampled ERP for condition A')
labelPlot(ax12[1], 'Original ERP for condition A')

In [ ]:
YouTubeVideo('feQk_vKloXk')
# NO CODE

**Step 3.** In the first two steps of the resampling procedure we created a single resampled ERP. Below, we define a function that performs these two calculations.

In [ ]:
def bootstrap_erp(eeg_data, n_samples=None):
    """ 
    Calculates bootstrap ERP from eeg data
    
    Parameters:
    -----------
        data: a ndarray, 'm' (# of trials) by 'n' 
              (# of time points), of voltage data.
        n_samples: an optional int of the number of
                   samples to return for `resampled_erp`.
    
    Returns:
    --------
        resampled_erp: a ndarray, 'n_samples' by 1, with 
                       the ERP of the resampled eeg data.
    """
    
    n_trials = len(eeg_data)  # get the number of trials
    if n_samples==None: # if `n_samples` is not specified, set as `n_trials`
        n_samples = n_trials
    i = np.random.randint(0, n_trials, n_samples)  # draw random trials
    eeg_0 = eeg_data[i]  # create resampled EEG
    resampled_erp = eeg_0.mean(0)  # calculate resampled ERP
    return resampled_erp

For step 3 of our bootstrapping process, we are instructed to repeat this procedure 3,000 times and create a distribution of ERPs. How can we do so? With a *for-loop*:

In [ ]:
# create a distribution of resampled ERPs
erp_0_dist = np.array([bootstrap_erp(eeg_a) for _ in range(3000)]) 

<div class="python-note">
    
Note that in the definition of `bootstrap_erp`, we included an argument `n_samples` that has a *default* value of `None`. This lets us assume that we want the resampled dataset to be the same size as the original, which is true for right now. Later, however, we will reuse this function but will not want the resampled data to be the same size as the original.
</div>

In [ ]:
YouTubeVideo('NLc93QESVZs')
# NO CODE

**Step 4.** In this step of the bootstrapping procedure, we determine for each time point the values greater than 2.5% and less than 97.5% of all values. There are many ways to perform this operation in Python, perhaps the easiest being to sort from smallest to largest the 3,000 resampled ERP values at each time point. With the resampled values sorted in this way, we then find the resampled ERP value at index 0.025 $\times$ 3000 = 75 and 0.975 $\times$ 3000 = 2925. These indices correspond to the resampled ERP values greater than 2.5% of all values and greater than 97.5% of all values, respectively, and therefore define the lower and upper confidence intervals at each moment in time. We can compute both confidence intervals in Python, and (at last!) plot the ERP for condition A with confidence intervals computed using the bootstrapping procedure: <a id="fig:1"></a>

In [ ]:
erp_0_dist.sort(axis=0)  # Sort each column of the resampled ERP
n = len(erp_0_dist)  # Define the number of samples
lower_ci = erp_0_dist[int(0.025*n)]  # Determine the lower CI
upper_ci = erp_0_dist[int(0.975*n)]  # Determine the upper CI

# plot erp_a and the bootstrap calculated CI bounds
fig13, ax13 = plt.subplots()
ax13.plot(t, erp_a, lw=2)
ax13.plot(t, lower_ci, 'k:')
ax13.plot(t, upper_ci, 'k:')  
ax13.hlines(0, 0, 1, 'r')
ax13.set_xticks(np.arange(0,1.05,0.05))

labelPlot(ax13, 'ERP of condition A with bootstrap confidence intervals') 

We can use these results to identify, for example, intervals in which the ERP differs significantly from zero by finding periods in which the confidence intervals do not include zero. The advantage of the bootstrapping procedure over other approaches is that this procedure requires few assumptions about the distribution of the statistic of interest, and that we use the observed data to probe the distribution of the statistic. The disadvantage of the bootstrapping procedure is that it is computationally intensive. Here we considered 3,000 resamplings, but we could easily consider more.

<div class="question">
    
**Q.** Compare the confidence intervals in the plot above (bootstrap confidence intervals) to [the CLT confidence intervals](#plt:erpA-m1) computed earlier. How are the two results similar or different? What happens to the confidence intervals if you change the number of resamplings in step 3 from 3,000 to 10,000?
</div>

__A__: The confidence interval computed here with the bootstrap is pretty similar to the one above computed using the CLT. Below we change the resamplings to 10,000. One should expect the CI bounds to get farther from the mean (we start with the real ERP and each successive sample perturbs it): this is what happens, but visually, there is no real difference between this plot and the one above.

In [ ]:
# Compute and plot bootstrapped ERP confidence intervals for
# a distribution of 10000 resampled ERPs

erp_0_dist = np.array([bootstrap_erp(eeg_a) for _ in range(10000)]) 

erp_0_dist.sort(axis=0)  # Sort each column of the resampled ERP
n = len(erp_0_dist)  # Define the number of samples
lower_ci = erp_0_dist[int(0.025*n)]  # Determine the lower CI
upper_ci = erp_0_dist[int(0.975*n)]  # Determine the upper CI

# plot erp_a and the bootstrap calculated CI bounds
fig14, ax14 = plt.subplots()
ax14.plot(t, erp_a, lw=2)
ax14.plot(t, lower_ci, 'k:')
ax14.plot(t, upper_ci, 'k:')  
ax14.hlines(0, 0, 1, 'r')
ax14.set_xticks(np.arange(0,1.1,0.1))

labelPlot(ax, 'ERP of condition A with bootstrap confidence intervals')

<div class="question">
    
**Q.** Compute the confidence intervals using the bootstrapping procedure for the ERP of condition B. What do you find?
</div>

__A__: First we'll create a function that 1) calculates a distribution of resampled ERPs and the 95% confidence interval bounds, and 2) plots the real ERP and the confidence intervals. Then we'll run the `eeg_b` data through this function.

In [ ]:
def plot_bootstrapped_erp(eeg_data, n=3000):
    """
    Plots confidence intervals for a distribution of resampled ERPs
    around the ERP of `eeg_data` and returns the distribution
    
    Parameters:
    -----------
        eeg_data: a ndarray (`n_trials` (# of trials) by `n_samples` 
                  (# of samples per trial)) of voltage data.
        n: an int of the number of resampled ERPs to compute
    
    Returns:
    --------
        erp_dist: a ndarray (`n` by `n_samples`) of a distribution
                  of ERPs of the resampled eeg data
        
    """
    
    # create a distribution of resampled ERPs
    erp = eeg_data.mean(0)
    erp_dist = np.array([bootstrap_erp(eeg_data) for _ in range(n)])
    
    erp_dist.sort(axis=0)  # Sort each column of the resampled ERP
    n_samples = eeg_data.shape[1]  # Define the number of samples
    lower_ci = erp_dist[int(0.025*n)]  # Determine the lower CI
    upper_ci = erp_dist[int(0.975*n)]  # Determine the upper CI

    # plot erp_a and the bootstrap calculated CI bounds
    fig, ax = plt.subplots()
    ax.plot(np.arange(0, n_samples, 1), erp, lw=2)
    ax.plot(np.arange(0, n_samples, 1), lower_ci, 'k:', lw=1)
    ax.plot(np.arange(0, n_samples, 1), upper_ci, 'k:', lw=1)  
    
    return erp_dist

In [ ]:
erp_dist_b = plot_bootstrapped_erp(eeg_b)

[Return to top](#introduction)

### A Bootstrap Test to Compare ERPs <a id="bootstrap"></a>

In [ ]:
YouTubeVideo('K6pgCxFdELc')
# NO CODE

The bootstrapping procedure provides a powerful technique to construct confidence intervals for the ERPs using only the observed EEG measurements. We can apply a similar technique to search for significant differences between the ERPs in conditions A and B. To do so, we first choose a *statistic*, a measure of some attribute of the difference between the two ERPs. There are many choices, some informative and some not. Let’s choose as our statistic the sum of the absolute difference in the ERPs across time from stimulus onset (t=0.25s) to 0.5 s after stimulus onset (t=0.75s). Computing this statistic is straightforward in Python:

In [ ]:
interval = np.array(t>=0.25) & np.array(t<=0.75)
stat = sum((np.abs(erp_a[interval]) - np.abs(erp_b[interval])))
print('stat = {:.4f}'.format(stat))

<div class="question">
    
**Q.** Given the value we determined for `stat`, are the ERPs for the two conditions different?
</div>

__A__: In isolation, the numerical value for `stat` is not very useful or interesting. Is the value for `stat` consistent with noisy scalp EEG data lacking an evoked response? Or is the value for `stat` large and unexpected to occur unless the ERPs in the two conditions are different? To make the statistic useful, we need `stat` to be interpretable, which we pursue here through a bootstrapping procedure. We assume that no difference exists between the two conditions; in the language of statistics, this is called the [*null hypothesis*](https://en.wikipedia.org/wiki/Null_hypothesis). If the null hypothesis holds, then we can pool all the EEG signals together from both conditions (for a total of 2,000 trials) and draw from this combined distribution to create resampled ERPs representative of either condition.

It may seem odd to create pseudodata by selecting trials across both conditions; intuitively, we may expect the data to differ in these two conditions and feel uncomfortable making a pseudodata set that includes trials from both conditions. But under the null hypothesis, we assume no difference between the EEG responses in conditions A and B, and we are therefore free to create pseudodata drawing from trials in both conditions. We do so with the goal of creating a distribution of values for `stat` under the null hypothesis that conditions A and B exhibit no difference. We then compare the observed value of `stat` with this distribution of `stat` values. If there is a difference between the two conditions, we expect to find the observed value of `stat` to be very different from the distribution of `stat` values generated from the pseudodata under the null hypothesis.

To create the distribution of `stat` values under the null hypothesis of no difference between the two conditions, we perform a bootstrap test. The idea is similar to the bootstrapping procedure used to construct the [confidence intervals for the ERP](#fig:1) <abbr class="figsup">fig<img src="imgs/2-1.png"></abbr>. We proceed as follows:

1. Merge the 1,000 trials each of EEG data from conditions A and B to form a combined distribution of 2,000 trials.
1. Sample with replacement 1,000 trials of EEG data from the combined distribution, and compute the resampled ERP.
1. Repeat step 2 and compute a second resampled ERP.
1. Compute the statistic, the maximum absolute value of the difference between the two resampled ERPs.
1. Repeat steps 2-4, 3,000 times to create a distribution of statistic values.
1. Compare the observed statistic to this distribution of statistic values. If the observed statistic is greater than 95% of the bootstrapped values, then reject the null hypothesis that the two conditions are the same.

The code to implement this procedure is similar to the bootstrapping procedure that we have already implemented to compute the confidence intervals for the ERP:

In [ ]:
YouTubeVideo('390ywma7S3U')
# NO CODE

In [ ]:
# Step 1

# Merge eeg data from both conditions
eeg_all = np.vstack((eeg_a, eeg_b))
# Seed our rng for reproducibility
np.random.seed(123)  

In [ ]:
# Steps 2-4

# define a function to sample with replacement 1,000 trials of `eeg_all`
# to create two resampled ERPs, then compute the statistic between them
def bootstrap_stat(eeg_data, t, n=1000, stim_on_time=0.25):
    """
    Computes a statistic (the sum of the absolute difference between ERPs 
    across time from stimulus onset to 0.5 s after stimulus onset, from 
    two computed ERPs, which are obtained by using the `bootstrap_eeg` 
    function
    
    Parameters:
    -----------
        eeg_data: a ndarray (`n_trials` (# of trials) by `n_samples` 
                  (# of samples per trial)) of voltage data.
        t: a ndarray (`n_samples` by 1) of the times of each samples
           for a trial
        n: an int of the number of resampled ERPs to compute
        stimon_time: a numeric value of the time, since the start of
                     a trial, at which the stimulus occurs
    
    Returns:
    --------
        stat: The statistic of interest
    """
    # Get two resampled ERPs.
    erp_1 = bootstrap_erp(eeg_all, n)
    erp_2 = bootstrap_erp(eeg_all, n)
    
    # Compute and return statistic:
    interval = np.array(t>=stim_on_time) & np.array(t<=(stim_on_time+0.5))
    stat = sum((np.abs(erp_1[interval]) - np.abs(erp_2[interval])))
    
    return stat

In [ ]:
# Step 5

# Create a distribution of 3,000 sample stats for `eeg_all` to
# compare to `stat`
stat_eeg_all = np.array([bootstrap_stat(eeg_all, t, 1000, 0.25) for _ in range(3000)])

In [ ]:
# Step 6

# Plot the distribution of `stat_eeg_all`, and a vetical line showing
# where `stat` would lie within this distribution
fig15, ax15 = plt.subplots()
h = ax15.hist(stat_eeg_all)
ax15.vlines(stat, 0, np.max(h[0]), 'r', label='`Stat` between ERPs of `eeg_a` and `eeg_b` datasets')
ax15.set_title('Empirical Distribution of `stat` for the Combined EEG Dataset')
ax15.set_xlabel('Value of `stat`')
ax15.set_ylabel('Count')
ax15.legend()
# Compute the empirical p-value for finding `stat` within `stat_eeg_all`
p_stat = sum(stat_eeg_all > stat) / len(stat_eeg_all)
print(
    'The probability of finding a value of `stat` within \n' \
    '`stat_eeg_all` is p=' + str(p_stat))

In [ ]:
YouTubeVideo('iefCPGHd5vY')
# NO CODE

<div class="question">
    
**Q.** Given the distribution of `statD` values shown above, and the value of `stat` computed from the original data, can we conclude that the difference between the two conditions is significant with this statistic?

**A.** Yes. Under the null hypothesis, the distribution fo the statistic ranges from approximately 0.15 to 0.33. The observed statistic `stat = 0.2884` exceeds most values in this distribution. Computing `sum(statD > stat)` we find in this example that only 18 of the 3,000 values in the distribution exceed the observed statistic. This corresponds to a proportion of 18/3000 = 0.006. We therefore reject the null hypothesis of no difference between the ERPs of conditions A and B. This result may be surprising, given how similar the two ERPs appear and the large variability in their differences ([see figure](#plt:differencedERP)).
</div>

This result illustrates the power of the bootstrapping procedure. We proposed a complicated statistic (the maximum absolute value of the difference between the two resampled ERPs). For this statistic, we do not possess an obvious formula to decide whether the resulting statistic is significant (we cannot rely on the CLT, for example). To determine significance, we employ a bootstrapping procedure (also known as a permutation test), which we can perform even for the relatively complicated statistic. In this way, we may devise complicated measures of data and construct error bars or compute statistical significance, provided our computational resources are sufficient.

[Return to top](#introduction)

## Summary <a id="summary"></a>

In this notebook, we considered scalp EEG data recorded from a single electrode during an auditory task. The task consisted of two conditions, and we sought to uncover the difference in the EEG responses between the two conditions. We began with a visual inspection of the EEG recordings from individual trials and from all trials, and concluded that the data were quite noisy; any evoked response due to the stimulus was not obvious in the single-trial data.

To emphasize the evoked signal, we computed the ERP, which involved averaging the EEG signal across trials. By doing so, we uncovered interesting structure in condition A, but not much in condition B. We then developed two techniques to add error bars to an ERP. One technique relied on the central limit theorem, and the other technique involved a computationally expensive bootstrapping procedure. Both techniques suggested that the ERP in condition A differed significantly from zero following the stimulus at time 0.25 s.

Finally, we assessed whether the two ERPs from condition A and condition B differed. We did so through visual inspection, by comparing the differences in the ERPs, and by computing a statistic and assessing its significance through a bootstrapping procedure. Using the last procedure, we concluded that the ERP in the two conditions significantly differed.

[Return to top](#introduction)